# Model 2: Tipus incidència 'Comentari problemàtic'

This notebook creates a evaluates a model to detect the issue 'Comentari problemàtic'.

It will be created a model to each language: catalan, spanish and english.

This type of issue is characterized by long coments composed for some sentences and imprecisse expressions.




In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy.util import minibatch, compounding

# Load English tokenizer, tagger, parser, NER and word vectors
#nlp = spacy.load("ca_fasttext_wiki")
#nlp = spacy.load("es_core_news_sm")
#nlp = spacy.load("en_core_web_sm")


In [4]:
# Creates tuples with texts and cats in the whole comment

def create_texts_and_cats_comments(data_com, nlp):
    
    # Converts data frame to list
    #data_com = data_com.dropna(subset="Comentari")
    data_com = data_com[data_com["Comentari"].notnull()]
    data_list = data_com["tuples"].tolist()
    
    # Change order of comments
    random.shuffle(data_list)
    if (debug >= 2):
        print ("Tuples list:")
        print (data_list[:5])  
        
    # Split text and label of true cases into two lists
    texts, cats = zip(*data_list)
    if (debug >= 2):
        print ("Texts:")
        print (texts[0:5])
        print ("Cats:")
        print (cats[0:5])  

    return (texts, cats)

In [5]:
# Creates tuples with texts and cats, spliting sentences in the comment

def create_texts_and_cats_sentences(data_com, nlp):
    
    # Select positive cases
    #data_true = data_com[data_com["label"]==True]
    #if (debug >= 1):
    #    print ("Data True cases:", data_true.shape[0])
    
    # Split comments in sentences
    data_com = data_com[data_com["Comentari"].notnull()]
    train_data = data_com["tuples"].tolist()
    train_list = []
    for row in train_data:
        comment = row[0]
        label = row[1]
        doc = nlp(comment)
        for sent in doc.sents:
            train_list.append((sent.text, label))
            if ((debug >=2) and (len(train_list)<=10)):
                print("+ ", sent)
                print("  ", label)
 
    if (debug >= 1):
        print("Sentences: ", len(train_list))

    # Change order of comments
    random.shuffle(train_list)
    if (debug >= 2):
        print ("Tuples list:")
        print (train_list[:5])  
        
     # Split text and label of true cases into two lists
    texts, cats = zip(*train_list)
    if (debug >= 2):
        print ("Texts:")
        print (texts[0:5])
        print ("Cats:")
        print (cats[0:5])   
        
    # Size of train_data_true and test_data_true
#    split_list = int(len(train_list) * split)+1
#    if (debug >= 1):
#        print ("Train sentences:", split_list, ", Test sentences: ", len(train_list)-split_list)   

    return (texts, cats)

In [19]:
# Loads information from preprocessed file to create de train and test data

def load_data(limit = 0, split = 0.8, language = "es", nlp = None):

    if (debug >= 1):
        print ("LOAD_DATA")
        print ("Language: ", language)
        
    # Load the model form spacy
#    nlp = spacy.load(model)
#    if (debug >= 1):
#        print ("Model loaded: ", model)
    
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print("Original data: ", data.shape[0])
        
    if (debug >= 2):
        print ("Sample:")
        display (data.sample(5))

    # Calculates label    
    data_com = data[["Comentari","TipusIncidencia"]]    # CHANGED
    if (debug >= 2):
        print ("Filtered columns:")
        display (data_com.sample(5))    

    # Calculates tuples row
    # Converts: label=True -> {"POSITIVE": True, "NEGATIVE": False}
    # label=False -> {"POSITIVE": False, "NEGATIVE": True}
    data_com["label"] = data_com["TipusIncidencia"] == "Comentari problemàtic"  # CHANGED
    data_com["tuples"] = data_com.apply(lambda row: (row["Comentari"], {"POSITIVE": bool(row["label"]), "NEGATIVE": not bool(row["label"])}), axis=1)    
    if (debug >= 2):
        print ("Tuples dataframe:")
        display (data_com.sample(5))

    # Create texts and cats lists for true cases
    data_true = data_com[data_com["label"]==True]
    if (debug >= 1):
        print ("True comments: ", data_true.shape[0])

    (texts_true, cats_true) = create_texts_and_cats_sentences(data_true, nlp)
    if (debug >= 2):
        print("Texts:",len(texts_true), "Cats:", len(cats_true))
        
    # Size of train_data_true and test_data_true
    split_true = int(len(texts_true) * split)+1
    if (debug >= 1):
        print ("Train True data:", split_true, ", Test True data: ", len(texts_true)-split_true)      
                
    # Create texts and cats lists for false cases        
    data_false = data_com[data_com["label"]==False]
    if (debug >= 1):
        print ("False comments:", data_false.shape[0])

    (texts_false, cats_false) = create_texts_and_cats_sentences(data_false, nlp)
    if (debug >= 2):
        print("Texts:",len(texts_false), "Cats:", len(cats_false))        
       
    # Size of train_data and test_data, considering limit
    if (limit > len(texts_true) + len(texts_false)):
        limit = len(texts_true) + len(texts_false)
    split_total = int(limit * split)

    if (debug >= 1):
        print ("Used data:", limit)
        print ("Train data: ", split_total, ", Test data: ", limit - split_total)
         
    # Size of train_data_false and test_data_false
    train_split_false = split_total - split_true
    test_split_false = (limit - split_total) - (len(texts_true) - split_true) 
        
    if (debug >= 1):
        print ("Used False comments: ", limit - len(texts_true))
        print ("Train False data: ", train_split_false, ", Test False data: ", test_split_false)  
        
         
    # Mix true and false cases and split in train and devel
    # All the true cases are included
    train_texts = texts_true[:split_true] + texts_false[:train_split_false]
    train_cats = cats_true[:split_true] + cats_false[:train_split_false]
    test_texts = texts_true[split_true:] + texts_false[-test_split_false:]
    test_cats = cats_true[split_true:] + cats_false[-test_split_false:]
        
    # Return train data and test data
    return (train_texts, train_cats), (test_texts, test_cats)

In [7]:
# Calculates the level of accuracy using the test set to compare the prediction with the label defined by the user 
# Obtains the indicators: Precission (p), Recall (r) and F-score (f)

def evaluate(tokenizer, textcat, texts, cats):
    fp_list = list()
    fn_list = list()
    
    if (debug == 2):
        print ("EVALUATE")
        
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
                if (debug >= 2):
                    print ("tp: ", doc)
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
                fp_list.append(doc)
                if (debug >= 2):
                    print ("fp: ", doc)
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
                if (debug >= 2):
                    print ("tn: ", doc)
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
                fn_list.append(doc)
                if (debug >= 2):
                    print ("fn: ", doc)   
    if (debug >= 2):
        print("tp: ", tp, ", fp: ", fp, ", tn: ", tn, ", fn: ", fn)
        
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
        
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score, 
            "tp": tp, "fp": fp, "tn": tn, "fn": fn,
            "fp_list": fp_list, "fn_list": fn_list}


In [8]:
# Loads the training information, transforms it, trains the model with this set of data, evaluate the result 
# and saves the resulting model

def train_model(model=None, language="ca", target_loss=0.001, n_texts=2000):
    if (debug >= 1):
        print ("TRAIN MODEL")
        
    # Load the model form spacy
    nlp = spacy.load(model)
    if (debug >= 1):
        print ("Model loaded: ", model)
        
    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe("textcat")

    # add label to text classifier
    textcat.add_label("POSITIVE")
    textcat.add_label("NEGATIVE")
    
    if "sentencizer" not in nlp.pipe_names:
        nlp.add_pipe(nlp.create_pipe('sentencizer'))
        
    # get names of other pipes to disable them during training
    pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec","sentencizer"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
#        if init_tok2vec is not None:
#            with init_tok2vec.open("rb") as file_:
#                textcat.model.tok2vec.from_bytes(file_.read())        
        
        if (debug >= 1):
            print ("Pipes prepared")
        
        # load training and test data
        (train_texts, train_cats), (dev_texts, dev_cats) = load_data(n_texts, 0.8, language, nlp)        

        if (debug >= 1):
            print(
                "Using {} examples ({} training, {} evaluation)".format(
                    len(train_texts)+len(dev_texts), len(train_texts), len(dev_texts)
                )
            )      

        # converts the data to the format:
        # (text, {'cats': {'POSITIVE': True, 'NEGATIVE': False}}))
        train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
        if (debug>=1):
            print ("Train_data: ")
            print (train_data[:5])        
    
        # Train the model
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}".format(
            "ITER", "LOSS", "P", "R", "F","TP","FP","FN","TN"))
        batch_sizes = compounding(4.0, 32.0, 1.001)
        loss = 100
        prev_loss = 1000
        iter = 1
#        if (loss > target_loss):
        while ((loss > target_loss) | (prev_loss - loss > target_loss)) & (iter <= 20):
            prev_loss = loss
    #        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)

#            if True:
#                batch = next(batches)
            for batch in batches:
                texts, annotations = zip(*batch)
                if (debug >= 2):
                    print ("Batch: ", len(batch))
                    print (batch)

                # Train the model discarting 20% of cases to avoid generalization
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
                if (debug >= 2):
                    print ("Model updated")

            # Evaluates model and print results
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print(
                "{0:.0f}\t{1:.3f}\t{2:.3f}\t{3:.3f}\t{4:.3f}\t {5:.0f}\t {6:.0f}\t {7:.0f}\t {8:.0f}".format(  # print a simple table
                    iter,
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                    scores["tp"],
                    scores["fp"],
                    scores["fn"],
                    scores["tn"]
                )
            )  
            loss = losses["textcat"]
            iter = iter + 1
            
        print("False positives:")
        display(scores["fp_list"][:10])
        print("False negatives:")
        display(scores["fn_list"][:5])
        
        print("Model trained")    
    
    # test the trained model
    print ("")
    print ("Some examples: ")
    test_texts ={"ca": ["Costa que pengi els materials acordats al campus",
                    "No he tingut aquesta professora",
                    "Sembla que tingui ganes de que patim, i ens tracta de tontos",
                    "No ha corregit examens fets a octubre, i estem al mes de gener.",
                    "No acabem d'entendre què és el que vol, però sembla que no va amb ella el tema"],
            "es": ["No he aprendido nada en ninguna de sus clases, ni siquiera he encontrado herramientas para buscarme la vida.", 
                   "Fue una de las maestras más sensatas y preocupadas por los estudiantes",
                   "Considero que no debería de seguir siendo profesor del inefc, puesto que ni tiene conocimientos ni ganas.",
                   "Hace que los alumnos vayan con muchas ganas a sus clases"
                   "Habla mal a los alumnos y se reprime a los que se quejan"
                  ],
            "en": ["I haven't had this professor",
                   "We don't know this person",
                   "He isn't a good professor",
                   "We had only a few lessons"
                  ]
        }     
    for test_text in test_texts[language]:
        doc = nlp(test_text)
        print(test_text, doc.cats)
    print ("")
    
    # Save de model
    with nlp.use_params(optimizer.averages):
        nlp.to_disk(pathmodel + language)
    print("Saved model to", pathmodel + language)
        
    # test the saved model
    print("Loading from", pathmodel+ language)
    nlp2 = spacy.load(pathmodel +  language)
    for test_text in test_texts[language]:
        doc2 = nlp2(test_text)
        print(test_text, doc2.cats)
    

In [9]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/model2/"
debug = 1

In [21]:
debug = 1
target_loss = 0.001
n_texts = 2000
#languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", "en":"en_core_web_sm" }
#languages = {"ca":"ca_fasttext_wiki_lg", "es":"es_core_news_lg", "en":"en_core_web_sm" }
languages = {"ca":"ca_fasttext_wiki"}

for language in languages:
    model = languages[language]
    
    print()
    #print ("TRAINING LANGUAGE: " + language)    
    #print ("Model: " + model)
    train_model (model, language, target_loss, n_texts)



TRAIN MODEL
Model loaded:  en_core_web_sm
Pipes prepared
LOAD_DATA
Language:  en
Original data:  236
True comments:  0
Sentences:  0


<ipython-input-19-ceccc245c8e3>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_com["label"] = data_com["TipusIncidencia"] == "Comentari problemàtic"  # CHANGED
<ipython-input-19-ceccc245c8e3>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_com["tuples"] = data_com.apply(lambda row: (row["Comentari"], {"POSITIVE": bool(row["label"]), "NEGATIVE": not bool(row["label"])}), axis=1)


ValueError: not enough values to unpack (expected 2, got 0)

In [17]:
# Test load_data function
debug = 2
n_texts = 2000
language = "ca"
model = "ca_fasttext_wiki"

# Load the model form spacy
nlp = spacy.load(model)
if (debug >= 1):
    print ("Model loaded: ", model)

# load training and test data
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(n_texts, 0.8, language, nlp)        


/home/david/.local/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'ca_fasttext_wiki' (0.0.2) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


KeyboardInterrupt: 